In [1]:
import sys
import copy
import torch
import numpy as np
import scipy as sp
import pandas as pd
import dill as pickle
import matplotlib.pyplot as plt

In [2]:
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")
from ronek2 import env
from ronek2 import const
from ronek2 import utils
from ronek2 import backend as bkd
from ronek2.systems import BoxAd
from ronek2.roms import CoBRAS

import ronek2.postproc.plotting as pltt

In [3]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": 1e-16,
  "floatx": "float64"
}
env.set(**env_opts)

In [4]:
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/ronek/examples/CR_Ar/database"

In [5]:
system = BoxAd(
  species={k: path_to_dtb + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p",
  use_coll_int_fit=True
)

In [6]:
system.compute_c_mat(max_mom=2, state_specs=False)

In [7]:
rho = 1e-2
T = 3e4
Te = 3e2
mu_quad = {
  "x": np.array([rho, T, Te]).reshape(1,-1),
  "w": np.ones((1,1))
}

In [8]:
cobras = CoBRAS(
  system=system,
  tgrid={"start": 1e-12, "stop": 1e-3, "num": 50},
  mu_quad=mu_quad,
  path_to_saving="./",
  saving=True
)

In [ ]:
X, Y = cobras(
  nb_meas=3,
  modes=False
)
X.shape, Y.shape

In [ ]:
cobras(
  X=X,
  Y=Y,
  xnot=[0,-3,-2,-1],
  modes=True,
  pod=True
)

In [11]:
bases = pickle.load(open("./cobras_bases.p", "rb"))

In [12]:
for i in range(15):
  nb = str(i+1)
  b = bases["phi"][:,i]
  pltt.plot_dist_2d(
    x=np.arange(len(b)),
    y=b,
    labels=[r"$\epsilon_i$ [eV]", r"$\%s_{%s}$" % ("phi", nb)],
    scales=["linear", "linear"],
    markersize=1,
    # figname=path + f"/{name}_{nb.zfill(2)}",
    save=False,
    show=False
  )

In [13]:
for i in range(15):
  nb = str(i+1)
  b = bases["psi"][:,i]
  pltt.plot_dist_2d(
    x=np.arange(len(b)),
    y=b,
    labels=[r"$\epsilon_i$ [eV]", r"$\%s_{%s}$" % ("psi", nb)],
    scales=["linear", "linear"],
    markersize=1,
    # figname=path + f"/{name}_{nb.zfill(2)}",
    save=False,
    show=False
  )

In [ ]:
t = system.get_tgrid(1e-13, 1e-3, 200)
y0, rho = system.equil.get_init_sol(mu_quad["x"].squeeze())
y0

In [15]:
yfom, _ = system.solve_fom(t, y0, rho)

In [16]:
w_fom = copy.deepcopy(yfom[:system.mix.nb_comp])
n_fom = system.mix.get_n(bkd.to_torch(w_fom)).numpy()
T_fom = copy.deepcopy(yfom[system.mix.nb_comp:])
T_fom[-1] = system.mix.get_Te(pe=T_fom[-1], ne=n_fom[-1])

In [29]:
rdim = 5

# bases = pickle.load(open("./pod_bases.p", "rb"))
# phi, psi = bases["phi"][:,:rdim], bases["phi"][:,:rdim]

bases = pickle.load(open("./cobras_bases.p", "rb"))
phi, psi = bases["phi"][:,:rdim], bases["psi"][:,:rdim]

system.set_basis(phi, psi)

In [ ]:
yrom, _ = system.solve_rom(t, y0, rho)
nt = len(yrom.T)
nt

In [31]:
w_rom = copy.deepcopy(yrom[:system.mix.nb_comp])
n_rom = system.mix.get_n(bkd.to_torch(w_rom)).numpy()
T_rom = copy.deepcopy(yrom[system.mix.nb_comp:])
T_rom[-1] = system.mix.get_Te(pe=T_rom[-1], ne=n_rom[-1])

In [ ]:
for i in range(2):
  plt.semilogx(t[1:nt], T_fom[i][1:nt])
  plt.semilogx(t[1:nt], T_rom[i][1:nt], ls="--", color=plt.gca().lines[-1].get_color())
plt.show()
plt.close()

In [33]:
for k in ("Ar", "Arp"):
  s = system.mix.species[k]
  pltt.plot_mom_evolution(
    path=f"./figs/{k}/",
    t=t[1:nt],
    n_m={"FOM": bkd.to_torch(n_fom.T[1:nt,s.indices]), "ROM": bkd.to_torch(n_rom.T[1:nt,s.indices])},
    molecule=s,
    molecule_label="{%s}" % k,
    tlim=None,
    ylim_err=None,
    err_scale="linear",
    hline=None,
    max_mom=2
  )

In [ ]:
for s in system.mix.species.values():
  print(s.name)
  for i in s.indices:
    plt.loglog(t[1:nt], n_fom[i][1:nt])
    plt.loglog(t[1:nt], n_rom[i][1:nt], ls="--", color=plt.gca().lines[-1].get_color())
  plt.show()
  plt.close()